# Scaling the Bodo Pipeline

<div class="alert alert-block alert-info">
<b>Warning:</b>
This notebook depends on the Parquet files generated by the notebook <b>01 Preparing the Data</b>. Make sure to run all cells in that notebook before executing this one.
    
In particular, the files required are:
<ul>
    <li><tt>ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2016_segmented.parquet</tt></li>
    <li><tt>ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2017_segmented.parquet</tt></li>
</ul>
</div>

This notebook follows on from the notebook **03 Constructing a Pipeline of Functions**. The principal distinction is the introduction of a way to *scale* the data. This allows us to make some timed experiments on various hardware configurations (that are available on [Bodo's cloud platform](https://platform.bodo.ai/)).

As usual, let's initialize an IPyParallel session (if needed).

In [1]:
# This cell does not need to be run on the Bodo.ai platform and should be skipped
import ipyparallel as ipp
import psutil; n_proc = min(psutil.cpu_count(logical=False), 8)
rc = ipp.Cluster(engines='mpi', n=n_proc).start_and_connect_sync(activate=True)

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|██████████| 4/4 [00:05<00:00,  1.50s/engine]


<div class="alert alert-block alert-info">
<b>Reminder:</b>

Having launched IPyParallel, the `px` magics ensure that code is executed on all engines.
    
1. The `%px` line magic executes a single line of code an all engines.
2. The `%%px` cell magic—placed as the first line of a cell—ensures all lines in a given cell are executed on all engines.
3. Lines executed outside of the context of a `px` line or cell magic execute in the namespace of the default Jupyter kernel.

Consult the [IPyParallel documentation](https://ipyparallel.readthedocs.io/en/latest/) for more information about setting up a local cluster. [Bodo's documentation](https://docs.bodo.ai/2022.2/bodo_parallelism/bodo_parallelism_basics/) provides more information about the parallel execution model.
</div>

In [2]:
%%px
# Some standard imports
import pandas as pd, numpy as np
import bodo, time

%px: 100%|██████████| 4/4 [00:01<00:00,  2.66tasks/s]


---

## A Tuned Pipeline

This is essentially the same pipeline from the notebook **03 Constructing a Pipeline of Functions**. There are two principal differences:

+ The function `load_parking_tickets` is replaced by a function `load_parking_tickets_scaled`. The original function produces a DataFrame with about 1.2 million rows; new function produces a DataFrame with $2^n$ times as many rows by concatenation (i.e., recursive doubling) for some input argument $n$.
+ Both the functions `load_parking_tickets_scaled` and `run_pipeline_scaled` accept an input parameter `n` (with default value `n=0`) to control the scale of the data operated on.

In [3]:
%%px

@bodo.jit
def load_parking_tickets_scaled(n=0):
    """
    Load data and aggregate by day, violation type, and police precinct.
    """

    start = time.time()
    groupby_cols = ['Issue Date','Violation County','Violation Precinct','Violation Code']
    DATA_SRC = 'ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2016_segmented.parquet'
    year_2016_df = pd.read_parquet(DATA_SRC)
    year_2016_df = year_2016_df.groupby(groupby_cols, as_index=False)['Summons Number'].count()

    DATA_SRC = 'ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2017_segmented.parquet'
    year_2017_df = pd.read_parquet(DATA_SRC)
    year_2017_df = year_2017_df.groupby(groupby_cols, as_index=False)['Summons Number'].count()

    # concatenate all dataframes into one dataframe
    many_year_df = pd.concat([year_2016_df, year_2017_df])
    
    #### recursively double number of rows in dataframe by concatenation; scales as 2**n
    for _ in range(n): # n=0 by default, so this loop is ignored generically
        many_year_df = pd.concat([many_year_df,many_year_df], ignore_index=True)
    
    end = time.time()
    timing_str = f"\n{'Reading Time:':<42}{end - start:8.3f} sec"
    return many_year_df, timing_str

In [4]:
%%px

@bodo.jit(distributed=False)
def load_violation_precincts_codes():
    """
    Load violation codes and precincts information.
    """
    start = time.time()
    violation_codes = pd.read_csv("ParkingData/DOF_Parking_Violation_Codes.csv")
    violation_codes.columns = ['Violation Code','Definition','manhattan_96_and_below','all_other_areas']
    nyc_precincts_df = pd.read_csv("ParkingData/nyc_precincts.csv", index_col='index')
    end = time.time()
    timing_str = f"\n{'Violation and precincts load Time:':<42}{end - start:8.3f} sec"
    return violation_codes, nyc_precincts_df, timing_str

In [5]:
%%px

@bodo.jit
def elim_code_36(main_df):
    """
    Remove undefined violations (code 36)
    """
    start = time.time()
    main_df = main_df[main_df['Violation Code']!=36].sort_values('Summons Number',ascending=False)
    end = time.time()
    timing_str = f"\n{'Eliminate undefined violations time:':<42}{end - start:8.3f} sec"
    return main_df, timing_str

In [6]:
%%px

@bodo.jit
def remove_outliers(main_df):
    """
    Delete entries that have dates outside our dataset dates
    """
    start = time.time()
    main_df = main_df[(main_df['Issue Date'] >= '2016-01-01') & (main_df['Issue Date'] <= '2017-12-31')]
    end = time.time()
    timing_str = f"\n{'Remove outliers time:':<42}{end - start:8.3f} sec"
    return main_df, timing_str

In [7]:
%%px

@bodo.jit
def merge_violation_code(main_df, violation_codes):
    """
    Merge violation information in the main_df
    """
    start = time.time()
    # left join main_df and violation_codes df so that there's more info on violation in main_df
    main_df = pd.merge(main_df, violation_codes, on='Violation Code', how='left')
    # cast precincts as integers from floats (inadvertent type change by merge)
    main_df['Violation Precinct'] = main_df['Violation Precinct'].astype(int)
    end = time.time()
    timing_str = f"\n{'Merge time:':<42}{end - start:8.3f} sec"
    return main_df, timing_str

In [8]:
%%px
@bodo.jit
def calculate_total_summons(main_df):
    """
    Calculate the total summonses in dollars for a violation in a precinct on a day
    """
    start = time.time()
    #create column for portion of precinct 96th st. and below
    n = len(main_df)
    portion_manhattan_96_and_below = np.empty(n, np.int64)
    # NOTE: To run Pandas, use this loop.
    # for i in range(n):
    for i in bodo.prange(n):
        x = main_df['Violation Precinct'].iat[i]
        if x < 22 or x == 23:
            portion_manhattan_96_and_below[i] = 1.0
        elif x == 22:
            portion_manhattan_96_and_below[i] = 0.75
        elif x == 24:
            portion_manhattan_96_and_below[i] = 0.5
        else: #other
            portion_manhattan_96_and_below[i] = 0
    main_df["portion_manhattan_96_and_below"] = portion_manhattan_96_and_below

    #create column for average dollar amount of summons based on location
    main_df['average_summons_amount'] = (main_df['portion_manhattan_96_and_below'] * main_df['manhattan_96_and_below']
                                     + (1 - main_df['portion_manhattan_96_and_below']) * main_df['all_other_areas'])

    #get total summons dollars by multiplying average dollar amount by number of summons given
    main_df['total_summons_dollars'] = main_df['Summons Number'] * main_df['average_summons_amount']
    main_df = main_df.sort_values(by=['total_summons_dollars'], ascending=False)
    end = time.time()
    timing_str = f"\n{'Calculate Total Summons Time:':<42}{end - start:8.3f} sec"
    return main_df, timing_str

In [9]:
%%px
@bodo.jit
def aggregate(main_df):
    '''function that aggregates and filters data
    e.g. total violations by precinct
    '''
    start = time.time()
    filtered_dataset = main_df[['Violation Precinct','Summons Number', 'total_summons_dollars']]
    precinct_offenses_df = filtered_dataset.groupby(by=['Violation Precinct']).sum().reset_index().fillna(0)
    end = time.time()
    timing_str = f"\n{'Aggregate code time:':<42}{end - start:8.3f} sec"
    return precinct_offenses_df, timing_str

---

## Executing the Pipeline



Having defined the functions above, we define a scalable pipeline function. This time, the pipeline accepts a parameter $n$ that controls the size of the computation (i.e., the number of rows is increased by a factor $2^n$ in the call to `load_parking_tickets`). 

In [10]:
%%px

@bodo.jit
def run_pipeline_scaled(n=0):
    start = time.time()
    main_df, out1 = load_parking_tickets_scaled(n)
    violation_codes, nyc_precincts_df, out2 = load_violation_precincts_codes()
    main_df, out3 = elim_code_36(main_df)
    main_df, out4 = remove_outliers(main_df)
    main_w_violation, out5 = merge_violation_code(main_df, violation_codes)
    total_summons, out6 = calculate_total_summons(main_w_violation)
    precinct_offenses_df, out7 = aggregate(total_summons)
    end = time.time()
    out8 = f'Execution time: (run_pipeline_scaled, {n=:1d})'
    out8 = f"\n{54*'='}\n{out8:<42}{end-start:8.3f} sec"
    output_str = ''.join(['\n', out1, out2, out3, out4, out5, out6, out7, out8, '\n'])
    return precinct_offenses_df, output_str

In [11]:
%%time
%%px
# Executing this cell with the %%time cell magic gives the total time.
# The first time run_pipeline_scaled is executed, all the functions are compiled.
# Notice "Wall time" > "Full Run" significantly
result, output = run_pipeline_scaled(n=3)
final_result = bodo.gatherv(result)
if bodo.get_rank()==0:
    display(final_result.head())
    print(final_result.shape)
    print(output)

%px:   0%|          | 0/4 [01:19<?, ?tasks/s]

[output:0]

,Violation Precinct,Summons Number,total_summons_dollars
0,19,6398768,558973960
1,104,1448224,95889680
2,115,2016360,131141200
3,43,1327672,92031880
4,10,1524392,141653400


[stdout:0] (283, 3)


Reading Time:                                4.434 sec
Violation and precincts load Time:           0.029 sec
Eliminate undefined violations time:         1.980 sec
Remove outliers time:                        0.080 sec
Merge time:                                  2.605 sec
Calculate Total Summons Time:                8.811 sec
Aggregate code time:                         0.333 sec
Execution time: (run_pipeline_scaled, n=3)  18.306 sec



%px: 100%|██████████| 4/4 [01:19<00:00, 19.94s/tasks]
CPU times: user 1.61 s, sys: 314 ms, total: 1.92 s
Wall time: 1min 21s


We notice that the time measured in the output only captures the execution time (i.e., over a minute `Wall time` versus about a quarter of a minute `Execution time` on this system). The very first time `run_pipeline_scaled` is executed, *all* the functions get compiled. This is noticeable at smaller scales when the `Execution time` is relatively small. Notice, however, that when we re-execute the cell—with the same parameter `n`—the discrepancy effectively vanishes.

In [12]:
%%time
%%px
# Executing this cell with the %%time cell magic gives the total time
# If run_pipeline_scaled has already been executed once, there is no compilation time overhead.
result, output = run_pipeline_scaled(n=3)
final_result = bodo.gatherv(result)
if bodo.get_rank()==0:
    display(final_result.head())
    print(final_result.shape)
    print(output)

%px:   0%|          | 0/4 [00:10<?, ?tasks/s]

[output:0]

,Violation Precinct,Summons Number,total_summons_dollars
0,19,6398768,558973960
1,104,1448224,95889680
2,115,2016360,131141200
3,43,1327672,92031880
4,10,1524392,141653400


[stdout:0] (283, 3)


Reading Time:                                3.457 sec
Violation and precincts load Time:           0.018 sec
Eliminate undefined violations time:         1.434 sec
Remove outliers time:                        0.078 sec
Merge time:                                  1.334 sec
Calculate Total Summons Time:                5.262 sec
Aggregate code time:                         0.337 sec
Execution time: (run_pipeline_scaled, n=3)  11.944 sec



%px: 100%|██████████| 4/4 [00:10<00:00,  2.52s/tasks]
CPU times: user 220 ms, sys: 54.7 ms, total: 274 ms
Wall time: 12.1 s


This time, all the compilation needed has already been done, so the discrepancy between the `Wall time` and the `Execution time` effectively vanishes (whatever minor differences remain reflect communication overheads & set-up costs and such). If we only need to run this code on a few million rows once, this compilation overhead is likely not needed. However, as our data scales, that compilation overhead diminishes quickly in a relative sense—the savings start to be well worth while.

---

## Timing Results

For the easiest apples-to-apples comparisons, here are some summaries using the pipeline functions from this notebook as well as those from the notebook **03 Constructing a Pipeline of Functions**.

On a workstation, executing `run_pipeline` yielded the following results:  

```
Reading Time:                              1.675 sec
Violation and precincts load Time:         0.009 sec
Eliminate undefined violations time:       0.050 sec
Remove outliers time:                      0.009 sec
Merge time:                                0.048 sec
Calculate Total Summons Time:              0.182 sec
Aggregate code time:                       0.057 sec
====================================================
Execution time (run_pipeline):             2.115 sec
```

Alternatively, the Bodo-compiled `run_pipeline_jitted` is able to make further improvements to overall run-time, in part thanks to keeping all data structures inside Bodo until the final return (i.e., avoiding the boxing/unboxing overhead).

```
Reading Time:                              1.132 sec
Violation and precincts load Time:         0.007 sec
Eliminate undefined violations time:       0.047 sec
Remove outliers time:                      0.004 sec
Merge time:                                0.060 sec
Calculate Total Summons Time:              0.196 sec
Aggregate code time:                       0.014 sec
====================================================
Execution time (run_pipeline_jitted):      1.461 sec
```


We present here timings when using much more data ($2^7=128$ times as much or roughly 140 million rows). We executed these on a Bodo cluster (2 c5.18xlarge, with 72 cores),

```
Reading Time:                                5.253 sec
Violation and precincts load Time:           0.091 sec
Eliminate undefined violations time:         2.971 sec
Remove outliers time:                        0.188 sec
Merge time:                                 16.986 sec
Calculate Total Summons Time:               45.737 sec
Aggregate code time:                         3.045 sec
======================================================
Execution time: (run_pipeline_scaled, n=7)  94.788 sec
```


```
Reading Time:                                6.101 sec
Violation and precincts load Time:           0.270 sec
Eliminate undefined violations time:         2.843 sec
Remove outliers time:                        0.057 sec
Merge time:                                 10.179 sec
Calculate Total Summons Time:               30.903 sec
Aggregate code time:                         3.286 sec
======================================================
Execution time: (run_pipeline_scaled, n=7)  53.789 sec
```

Remember to cleanly shut down the `ipyparallel` session:

In [13]:
# To stop the cluster run the following command.
rc.cluster.stop_cluster_sync()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 2082, 'identifier': 'ipcontroller-1651429857-cjjr-2059'}
Stopping engine(s): 1651429858
engine set stopped 1651429858: {'exit_code': 0, 'pid': 2098, 'identifier': 'ipengine-1651429857-cjjr-1651429858-2059'}


---